<a href="https://colab.research.google.com/github/JEN6YT/APS360-Project/blob/main/Primary_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from PIL import Image, ImageDraw
import pandas as pd
import os
import torchvision.transforms as transforms
import torch
import random
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Get image and label


In [6]:
# read the xlsx file from Google Drive
file_path = '/content/drive/My Drive/U of T/APS360 Deep Learning/NIH-NLM-ThinBloodSmearsPf/img_path.xlsx'

# The dataframe store all the path, i.e. 'NIH-NLM-ThinBloodSmearsPf\Polygon Set\142C38P...'
df1 = pd.read_excel(file_path)

# create complete path for each image
prefix = "/content/drive/My Drive/U of T/APS360 Deep Learning/"
df1 = df1.applymap(lambda x: prefix + str(x))


In [ ]:
df1.head()

,File path
0,/content/drive/My Drive/U of T/APS360 Deep Lea...
1,/content/drive/My Drive/U of T/APS360 Deep Lea...
2,/content/drive/My Drive/U of T/APS360 Deep Lea...
3,/content/drive/My Drive/U of T/APS360 Deep Lea...
4,/content/drive/My Drive/U of T/APS360 Deep Lea...


In [7]:
df1.shape

(800, 1)

In [8]:
# read the xlsx file from Google Drive
file_path_infected = '/content/drive/My Drive/U of T/APS360 Deep Learning/NIH-NLM-ThinBloodSmearsPf/infected_RBC.xlsx'

# The dataframe store all the path, i.e. 'NIH-NLM-ThinBloodSmearsPf\Polygon Set\142C38P...'
df2 = pd.read_excel(file_path_infected)

In [ ]:
df2.head()

,Infected RBC
0,39
1,27
2,42
3,37
4,45


In [9]:
df = pd.merge(df1, df2, left_index=True, right_index=True)

In [10]:
df.head()

,File path,Infected RBC
0,/content/drive/My Drive/U of T/APS360 Deep Lea...,39
1,/content/drive/My Drive/U of T/APS360 Deep Lea...,27
2,/content/drive/My Drive/U of T/APS360 Deep Lea...,42
3,/content/drive/My Drive/U of T/APS360 Deep Lea...,37
4,/content/drive/My Drive/U of T/APS360 Deep Lea...,45


In [11]:
df.shape

(800, 2)

## Label, resize, save in array

In [12]:
data_list = []
target_size = (224, 224)

# iterate over rows and label images
for i, row in df.iterrows():
  # get the file path and infected RBC count
  print(i)

  file_path = row['File path']
  infected_rbc = row['Infected RBC']
  
  # open the image using PIL
  image = Image.open(file_path)
  image = image.resize(target_size)
  img_arr = np.array(image)
  
  # label the image based on the infected RBC count
  if infected_rbc == 0:
    label = 0
  elif infected_rbc > 0:
    label = 1
  
  # append the image and label to the list
  data = {'image': img_arr, 'label': label}
  data_list.append(data)

# convert the list to numpy array and save it
np.save('data.npy', np.array(data_list))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
data_list[0]

{'image': array([[[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 2],
         [1, 1, 3],
         [1, 1, 2]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [0, 0, 1],
         [1, 1, 1],
         [1, 1, 2]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [0, 0, 1],
         [1, 1, 1]],
 
        ...,
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],
 
        [[0, 0, 0],
         [1, 1, 1],
         [1, 0, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1],
         ...,
         [1, 1, 1],
         [1, 1, 1],
         [2, 0, 1]]], dtype=uint8), 'label': 1}

# Checking balance

In [14]:
infected = 0
uninfected = 0
for data in data_list:
  if data['label'] == 1:
    infected += 1
  if data['label'] == 0:
    uninfected += 1
total = infected+uninfected
print(f'infected data {infected/total} and uninfected data {uninfected/total}')

infected data 0.71125 and uninfected data 0.28875


## Resampling

In [15]:
# load the data from the saved numpy array
data = np.load('data.npy', allow_pickle=True)

# get the features (images) and labels
image = np.array([d['image'].flatten() for d in data])
label = np.array([d['label'] for d in data])

# reshape X to a 2D array
image = image.reshape(image.shape[0], -1)

# apply RandomOverSampler to X and y
ros = RandomOverSampler()
image_resampled, label_resampled = ros.fit_resample(image, label)

# reshape X_resampled back to 4D array
image_resampled = image_resampled.reshape(image_resampled.shape[0], 224, 224, 3)

# combine X_resampled and y_resampled into a list of dicts
data_resampled = [{'image': image_resampled[i], 'label': label_resampled[i]} for i in range(len(label_resampled))]

# save the resampled data to a new numpy array
np.save('data_resampled.npy', np.array(data_resampled))

In [16]:
infected = 0
uninfected = 0
for data in data_resampled:
  if data['label'] == 1:
    infected += 1
  if data['label'] == 0:
    uninfected += 1
total = infected+uninfected
print(f'infected data {infected/total} and uninfected data {uninfected/total}')

infected data 0.5 and uninfected data 0.5


In [17]:
total

1138

# Normalizing

In [18]:
# Define the normalization function
def normalize(image):
  return (image - np.min(image)) / (np.max(image) - np.min(image))

# Normalize the data
normalized_data = []
for data in data_resampled:
  normalized_image = normalize(data['image'])
  normalized_data.append({'image': normalized_image, 'label': data['label']})

# Save the normalized data as a .npy file
np.save('normalized_data.npy', normalized_data)

# Splitting

In [19]:
# get the features (images) and labels
X = np.array([d['image'] for d in normalized_data])
y = np.array([d['label'] for d in normalized_data])

# Split your data into training, validation and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=66)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=66)

In [20]:
import torch
X_train  = torch.from_numpy(X_train)
y_train  = torch.from_numpy(y_train)
X_val = torch.from_numpy(X_val)

In [76]:
print(len(X_train),len(X_val),len(X_test))

637 273 228


In [97]:
X_train.shape

torch.Size([637, 224, 224, 3])

# Primary Model



Primary Model: U-net

In Image Segmentation, the machine has to partition the image into different segments, each of them representing a different entity.

As you can see above, how the image turned into two segments, one represents the cat and the other background.

In image segmentation, we not only need to convert feature map into a vector but also reconstruct an image from this vector.

**Encoder/down -> bottleneck -> Decoder/up**

The contraction section is made of **many contraction blocks**. Each block takes an input applies two 3X3 convolution layers followed by a 2X2 max pooling. The **number of kernels or feature maps after each block doubles** so that architecture can learn the complex structures effectively. The bottommost layer mediates between the contraction layer and the expansion layer. It uses two 3X3 CNN layers followed by 2X2 up convolution layer.

But the heart of this architecture lies in **the expansion section**. Similar to contraction layer, it also consists of several expansion blocks. Each block passes the input to two 3X3 CNN layers followed by a 2X2 upsampling layer. Also after **each block number of feature maps used by convolutional layer get half to maintain symmetry**. However, every time the input is also get appended by feature maps of the corresponding contraction layer. This action would ensure that the features that are learned while contracting the image will be used to reconstruct it. The number of expansion blocks is as same as the number of contraction block. After that, the resultant mapping passes through another 3X3 CNN layer with the number of feature maps equal to the number of segments desired.

In [21]:
# import Pytorch Library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# import matplotlib lirary
import matplotlib.pyplot as plt

In [22]:
# create a double convolutional layer
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

# two convolutional layers followed by max pooling for condensing
class Down(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

# The intuition is that we would like to restore the condensed feature map to the original size of the input image
# therefore we expand the feature dimensions. Upsampling is also referred to as transposed convolution
# here, we are using bilinear interpolation to upsample
# skip connection
class Up(nn.Module):
    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

# Unet model
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super().__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)

        # four encoder blocks
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        # four decoder blocks
        self.up1 = Up(1024, 512, bilinear)
        self.up2 = Up(512, 256, bilinear)
        self.up3 = Up(256, 128, bilinear)
        self.up4 = Up(128, 64, bilinear)
        # pass it to a convolutional layer for output
        #self.outc = nn.Conv2d(64, n_classes, kernel_size=1)
        
        self.outc = nn.Conv2d(64, n_classes, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits


In [23]:
def get_accuracy(model, data):
    correct, total = 0, 0
    for sms, labels in data:
        output = model(sms[0])
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += labels.shape[0]
    return correct / total

In [24]:
def train_network(model, train, valid, num_epochs=5, batch_size = 64, learning_rate=1e-5):
    optimizer = optim.Adam(model.parameters(), learning_rate)
    criterion = nn.CrossEntropyLoss()

    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

    val_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=True)
    

    losses, train_acc, valid_acc = [], [], []
    epochs = []

    for epoch in range(num_epochs):
        for image in iter(train_loader):
            optimizer.zero_grad()
            model.float()
            pred = model(image)
            loss = criterion(pred, image)
            loss.backward()
            optimizer.step()
        
        losses.append(float(loss))
        epochs.append(epoch+1)
        train_acc.append(get_accuracy(model, train_loader))
        valid_acc.append(get_accuracy(model, val_loader))
        print("Epoch %d; Loss %f; Train Acc %f; Val Acc %f" % (
              epoch+1, loss, train_acc[-1], valid_acc[-1]))
    # plotting
    plt.title("Training Curve")
    plt.plot(losses, label="Train")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(epochs, train_acc, label="Train")
    plt.plot(epochs, valid_acc, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

In [62]:
#unet = UNet(3,2)

train_network(unet, X_train, X_val)

RuntimeError: ignored

In [63]:
unet.inc(X_train)

RuntimeError: ignored

In [59]:
type(X_train)

torch.Tensor

In [34]:
np.shape(X_train)

torch.Size([637, 224, 224, 3])

In [64]:
type(nn.ReLU)

type